# 언어 모델

In [1]:
!nvidia-smi

Wed Dec 21 09:49:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers sentencepiece tokenizers sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 7.3 MB/s 
     |████████████████████████████████| 1.3 MB 65.3 MB/s 
     |████████████████████████████████| 7.6 MB 52.2 MB/s 
     |████████████████████████████████| 85 kB 6.2 MB/s 
     |████████████████████████████████| 182 kB 44.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=2a7414c46990c9bcc620d523883610e1af6881ec493bc90804f5282ac1381db3
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [3]:
from transformers import pipeline

## 긍정 부정

In [4]:
model = pipeline(
    'text-classification',
    model='jaesun/kcbert-base-finetuned-nsmc',
    device='cuda:0'
)

Downloading:   0%|          | 0.00/883 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/370 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [8]:
model.model.config.id2label = {
    0: '부정',
    1: '긍정'
}

In [9]:
#text1 = "이 영화 꿀잼인데 꼭 보세요"
text2 = "이걸 보고 기분이 매우 좋지 않지 않지 않지 않았다."

out = model(text2)
print(out)

[{'label': '부정', 'score': 0.9865174889564514}]


## 혐오 표현

In [15]:
model = pipeline(
    'text-classification',
    model='jason9693/soongsil-bert-base-apeach'
)

In [41]:
text1 = '로보토미'

out = model([text1])
print(out)

[{'label': 'IT과학', 'score': 0.864922046661377}]


## Topic Classification

In [40]:
labels = ["IT과학", "경제", "사회", "생활문화", "세계", "스포츠", "정치"]

model = pipeline(
    'text-classification',
    model='jihoonkimharu/bert-base-klue-ynat-finetuned'
)
model.model.config.id2label = dict(enumerate(labels))

Downloading:   0%|          | 0.00/995 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/563 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [50]:
#print(model("러시아 우크라이나 침공"))
#print(model("미국연준 기준금리 대폭인하 증시충격...!"))
#print(model("나로호 발사 또 실패 국민 세금 언제까지 낭비할건가"))
print(model("프랑스'루브르 아부다비에 있다' 전 세계에 충격을 줘"))

[{'label': '세계', 'score': 0.988068163394928}]


In [52]:
model = pipeline(
    'text-classification',
    model='Huffon/klue-roberta-base-nli'
)

Downloading:   0%|          | 0.00/926 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [53]:
model(
    [
        "배고프다 [SEP] 오늘 저녁은 뭐야?", #중립(연관 없음)
        "배고프다 [SEP] 밥 먹고 싶은데", #수반(연관)
        "배고프다 [SEP] 배 안고픈데", #모순(상반됨)
    ]
)

[{'label': 'NEUTRAL', 'score': 0.9920761585235596},
 {'label': 'ENTAILMENT', 'score': 0.9320499300956726},
 {'label': 'CONTRADICTION', 'score': 0.9984579086303711}]

In [61]:
model(
    ["결국 그녀와 헤어졌다 [SEP] 여름이었다"]
)

[{'label': 'NEUTRAL', 'score': 0.9990158081054688}]

## NLI를 이용한 zero-shot 분류


*   zero-shot : 학습에 단 하나의 데이터도 사용하지 않았음
*   one-shot : 학습에 단 하나의 데이터만 사용
*   few-shot : 학습에 2개 이상이고 적은 데이터 사용




In [75]:
model(
    ["망치로 써도 될 정도의 폰임 [SEP] 노키아 폰"]
)

[{'label': 'NEUTRAL', 'score': 0.9983934760093689}]

In [81]:

model = pipeline(
    "zero-shot-classification",
    model = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
)

In [82]:
model(
    "아이폰 작동이 안됨",
    ["고장", "배송", "환불", "예약"]
)

{'sequence': '아이폰 작동이 안됨',
 'labels': ['고장', '환불', '예약', '배송'],
 'scores': [0.716954231262207,
  0.09861079603433609,
  0.0946943387389183,
  0.08974058926105499]}

## 새 섹션

In [88]:
model = pipeline(
    "token-classification",
    model = "MrBananaHuman/klue_ner",
    tokenizer="klue/roberta-base"
)

Downloading:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [89]:
ner_labels = ["B-DT","I-DT","B-LC","I-LC","B-OG","I-OG","B-PS","I-PS","B-QT","I-QT","B-TI","I-TI", "no"]
model.model.config.id2label = dict(enumerate(ner_labels))

In [ ]:
model("나는야 퉁퉁이~ 3년째 마을 골목대장을 맡고있지롱")

In [91]:
def filter_ner(ner_result):
  prev_tokens=""
  prev_tag=""

  entities = []

  for item in ner_result:
    if item["entity"] == "no":
      continue

    tag = item["entity"]
    print(tag, item)

    if tag[0] == "B":
      if prev_tag and prev_tokens:
        entities.append((prev_tag, prev_tokens))
      prev_tokens = item["word"]
    elif item["word"].startswith("##"):
      prev_tokens += item["word"][2:]
    else:
      prev_tokens += " " + item["word"]
    prev_tag = tag[2:]

  entities.append((prev_tag, prev_tokens))
  print(entities)

ner_result = model(
    "내 이름은 희규고 경기도 일산 동구에서 삼년째 살고 있어요. 세시간 뒤 선린인터넷 고등학교에 방문하려고요.")

filter_ner(ner_result)

B-PS {'entity': 'B-PS', 'score': 0.9711964, 'index': 4, 'word': '희', 'start': 6, 'end': 7}
I-PS {'entity': 'I-PS', 'score': 0.9866154, 'index': 5, 'word': '##규', 'start': 7, 'end': 8}
B-LC {'entity': 'B-LC', 'score': 0.9964461, 'index': 7, 'word': '경기도', 'start': 10, 'end': 13}
I-LC {'entity': 'I-LC', 'score': 0.98236626, 'index': 8, 'word': '일산', 'start': 14, 'end': 16}
I-LC {'entity': 'I-LC', 'score': 0.9901938, 'index': 9, 'word': '동구', 'start': 17, 'end': 19}
B-DT {'entity': 'B-DT', 'score': 0.9905157, 'index': 11, 'word': '삼', 'start': 22, 'end': 23}
I-DT {'entity': 'I-DT', 'score': 0.99319273, 'index': 12, 'word': '##년', 'start': 23, 'end': 24}
I-DT {'entity': 'I-DT', 'score': 0.64103353, 'index': 13, 'word': '##째', 'start': 24, 'end': 25}
B-TI {'entity': 'B-TI', 'score': 0.9931028, 'index': 19, 'word': '세', 'start': 34, 'end': 35}
I-TI {'entity': 'I-TI', 'score': 0.99616796, 'index': 20, 'word': '##시간', 'start': 35, 'end': 37}
I-TI {'entity': 'I-TI', 'score': 0.6585532, 'index':

## 질문

In [92]:
model = pipeline(
    'question-answering',
    model="monologg/koelectra-base-v3-finetuned-korquad",
    handle_impossible_answer=True
)

Downloading:   0%|          | 0.00/591 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/449M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/111 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [93]:
model(
  question="조선 중기의 무신인 이순신이 태어난 날짜는?",
  context="이순신(李舜臣, 1545년 4월 28일 (음력 3월 8일) ~ 1598년 12월 16일 (음력 11월 19일))은 조선 중기의 무신이었다. 본관은 덕수(德水), 자는 여해(汝諧), 시호는 충무(忠武)였으며, 한성 출신이었다. 문반 가문 출신으로 1576년(선조 9년) 무과(武科)에 급제[2]하여 그 관직이 동구비보 권관, 훈련원 봉사, 발포진 수군만호, 조산보 만호, 전라좌도수사를 거쳐 정헌대부 삼도수군통제사에 이르렀다."
)

{'score': 0.1668979972600937,
 'start': 35,
 'end': 48,
 'answer': '1598년 12월 16일'}

In [94]:
model(
  question="손흥민의 나이는?",
  context="손흥민(孫興慜, 1992년 7월 8일 ~ )은 대한민국의 축구 선수이다. 현재 잉글랜드 프리미어리그 토트넘 홋스퍼에서 윙어로 활약하고 있으며, 대한민국 축구 국가대표팀의 주장으로도 활동 중이다. 영국에서는 애칭인 \"쏘니\"(Sonny)로 불린다. 아시아 선수로서는 역대 최초로 프리미어리그 공식 베스트 일레븐에 선정됐고, 프리미어리그 득점왕을 수상했다. 또한 FIFA 푸스카스상을 대한민국 선수 최초로 수상하였으며 2022년에는 축구 선수로는 최초로 대한민국 체육훈장 청룡장을 서훈받았다."
)


{'score': 0.995119035243988, 'start': 9, 'end': 20, 'answer': '1992년 7월 8일'}

In [98]:
model(
  question="파이썬의 모티프는?",
  context="창시자는 네덜란드의 프로그래머 귀도 반 로섬(Guido van Rossum).[2] [3]1989년 크리스마스 주에, 연구실이 닫혀있어서 심심한 김에 만든 프로그래밍 언어이다. 농담이 아니고 반 로섬을 유럽에서는 애덤 스미스에 비교할 정도며, 네덜란드에서는 기술자의 대명사로 취급된다. 프로그래밍계의 경제학자라나... 심심해서 만들었다는 것은 파이썬 서문과 마이크로소프트웨어와 한 인터뷰를 보면 알겠지만 사실이다. 능력 있는 기술자들은 대부분 심심할 때, 혹은 실수로 걸작을 만든다. 그리고 파이썬이라는 이름은 귀도가 즐겨 보던 영국의 6인조 코미디 그룹 몬티 파이썬에서 따왔다고 한다.[4]"
)

{'score': 0.9999921321868896, 'start': 311, 'end': 319, 'answer': '몬티 파이썬에서'}

In [99]:
model = pipeline(
    'text2text-generation',
    model='paust/pko-t5-base-finetuned-korquad'
    
)

Downloading:   0%|          | 0.00/728 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.41G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/209 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.90M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [100]:
def answer_question(question, title, context):
  input = f"question: {question} title: {title} context: {context}"
  print(model(input))

answer_question(
  "조선 중기의 무신인 이순신이 태어난 날짜는?",
  "이순신",
  "이순신(李舜臣, 1545년 4월 28일 (음력 3월 8일) ~ 1598년 12월 16일 (음력 11월 19일))은 조선 중기의 무신이었다. 본관은 덕수(德水), 자는 여해(汝諧), 시호는 충무(忠武)였으며, 한성 출신이었다. 문반 가문 출신으로 1576년(선조 9년) 무과(武科)에 급제[2]하여 그 관직이 동구비보 권관, 훈련원 봉사, 발포진 수군만호, 조산보 만호, 전라좌도수사를 거쳐 정헌대부 삼도수군통제사에 이르렀다."
)


[{'generated_text': '1545년 4월 28일'}]


In [101]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

Downloading:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/707 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/967k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/394 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/336k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [102]:
sentences = ['잠이 옵니다', '졸음이 옵니다', '기차가 옵니다']
vectors = model.encode(sentences)
print(vectors, vectors.shape)

[[ 0.05224039 -1.0250845   0.05070774 ... -0.3858475  -1.4207256
  -1.062261  ]
 [ 0.6302679  -0.6792767  -0.7553818  ... -0.08747681  0.2238219
  -0.7263258 ]
 [ 0.10370272  0.04884144  0.5257997  ... -0.8497961  -1.0043038
  -0.64980704]] (3, 768)


In [103]:
similarities = util.cos_sim(vectors, vectors)
print(similarities)

tensor([[1.0000, 0.6577, 0.2732],
        [0.6577, 1.0000, 0.2730],
        [0.2732, 0.2730, 1.0000]])
